**Text Classification with Hugging Face Transformers in ktrain**

PoC  by : sreejithc321

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

Data Preperation

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
train_b = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',categories=categories, shuffle=True, random_state=42)

print('size of training set: %s' % (len(train_b['data'])))
print('size of testing set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 2257
size of testing set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


**Model = distilbert-base-uncased**

In [ ]:
! pip install ktrain
import ktrain
from ktrain import text

MODEL_NAME = 'distilbert-base-uncased'

t = text.Transformer(MODEL_NAME, maxlen=500, class_names=train_b.target_names)

trn = t.preprocess_train(x_train, y_train)

val = t.preprocess_test(x_test, y_test)

model = t.get_classifier()

learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

     |████████████████████████████████| 25.3MB 169kB/s 
     |████████████████████████████████| 983kB 44.8MB/s 
     |████████████████████████████████| 245kB 46.2MB/s 
     |████████████████████████████████| 1.0MB 42.8MB/s 
     |████████████████████████████████| 471kB 43.6MB/s 
     |████████████████████████████████| 1.1MB 41.8MB/s 
     |████████████████████████████████| 890kB 43.1MB/s 
     |████████████████████████████████| 3.0MB 46.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.21.3-cp36-none-any.whl size=25270203 sha256=c1c01a893ae7ff47711b6ad32fcb05844d3c575e67286e36ae82d90918bc6023
  Stored in directory: /root/.cache/pip/wheels/d6/ef/6d/9428365b27ac1e84a77214c5718e7f8b2180007af593791247
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=b07f729c27f88b595f9bf435825de3fc788e3579513b307f6988616153765a80
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan


preprocessing train...
language: en
train sequence lengths:
	mean : 308
	95percentile : 837
	99percentile : 1938


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 343
	95percentile : 979
	99percentile : 2562


In [ ]:
#learner.lr_find(show_plot=True, max_epochs=2)

Using 1 epoch

In [ ]:
learner.fit_onecycle(8e-5, 1)



begin training using onecycle policy with max lr of 8e-05...
377/377 [==============================] - 7957s 21s/step - loss: 0.4630 - accuracy: 0.8370 - val_loss: 0.3070 - val_accuracy: 0.9028


In [ ]:
learner.validate(class_names=t.get_classes())

                        precision    recall  f1-score   support

           alt.atheism       0.83      0.77      0.80       319
         comp.graphics       0.97      0.92      0.94       389
               sci.med       0.96      0.93      0.94       396
soc.religion.christian       0.85      0.96      0.90       398

              accuracy                           0.90      1502
             macro avg       0.90      0.90      0.90      1502
          weighted avg       0.90      0.90      0.90      1502



array([[246,   1,   8,  64],
       [ 20, 359,   7,   3],
       [ 16,  11, 369,   0],
       [ 13,   1,   2, 382]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
predictor.predict('Jesus Christ is the central figure of Christianity.')

'soc.religion.christian'

In [ ]:
predictor.explain('Jesus Christ is the central figure of Christianity.')

In [ ]:
predictor.predict('medicine are essential for curing pandemics')

'sci.med'